In [1]:
%run ./MovieLens_Ranking.ipynb

In [2]:
# 设置超参数
k = 5
lr = 0.01
epochs = 500
d = 20
alpha_u = alpha_v = beta_v = 0.01

In [3]:
# 获得用户不感兴趣的列表
items_not_like = [[]]
for u in range(1, user_num + 1):
    items_not_like.append(I_not_u(u))

def get_item_not_like(u):
    items = items_not_like[u]
    return random.choice(items)

In [4]:
# 初始化模型参数
b = np.zeros(item_num + 1, float)
for i in range(1, item_num + 1):
    b[i] = y_ui[:, i].mean() - density
U = np.random.rand(user_num + 1, d)
V = np.random.rand(item_num + 1, d)
U = (U - 0.5) * 0.01
V = (V - 0.5) * 0.01

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 预测函数
def predict_rule(u, i, U, V, b):
    return U[u] @ V[i].T + b[i]

In [6]:
# 训练模型
for epoch in tqdm(range(epochs)):
    # 打乱训练集
    train_data = get_train_data()
    random.shuffle(train_data)
    for data in train_data:
        # 随机取数据
        u, i = data[0], data[1]
        j = get_item_not_like(u)
        # 计算梯度
        r_ui, r_uj = predict_rule(u, i, U, V, b), predict_rule(u, j, U, V, b)
        e = -sigmoid(-(r_ui - r_uj))
        delta_Uu = e * (V[i] - V[j]) + alpha_u * U[u]
        delta_Vi = e * U[u] + alpha_v * V[i]
        delta_Vj = e * (-U[u]) + alpha_v * V[j]
        delta_bi = e + beta_v * b[i]
        delta_bj = -e + beta_v * b[j]
        # 更新参数
        U[u] -= lr * delta_Uu
        V[i] -= lr * delta_Vi
        V[j] -= lr * delta_Vj
        b[i] -= lr * delta_bi
        b[j] -= lr * delta_bj

100%|█████████████████████████████████████████| 500/500 [05:34<00:00,  1.49it/s]


In [7]:
def BPR(u, j):
    return U[u] @ V[j].T + b[j]

In [8]:
predict(BPR, k)

Pre: 0.3820
Rec: 0.1163
